### Importing libraries

In [1]:
import numpy as np
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from keras import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.models import * 
from tensorflow.keras.preprocessing import image

## Image data generation

In [2]:
train_path= '../input/ultrasound-breast-images-for-breast-cancer/ultrasound breast classification/train'
test_path='../input/ultrasound-breast-images-for-breast-cancer/ultrasound breast classification/val'

In [3]:
train_datagen = image.ImageDataGenerator(
    rotation_range=15,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    width_shift_range=0.1,
    height_shift_range=0.1
)
val_datagen= image.ImageDataGenerator(    rotation_range=15,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    width_shift_range=0.1,
    height_shift_range=0.1)

In [5]:
train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size = (224,224),
    batch_size = 4,
    class_mode = 'binary')
validation_generator = val_datagen.flow_from_directory(
    test_path,
    target_size = (224,224),
    batch_size = 4,
    shuffle=True,
    class_mode = 'binary')

Found 8116 images belonging to 2 classes.
Found 900 images belonging to 2 classes.


## CNN Model- EfficientNetB3

In [6]:
base_model = tf.keras.applications.EfficientNetB3(weights='imagenet', input_shape=(224,224,3), include_top=False)

for layer in base_model.layers:
    layer.trainable=False
model = Sequential()
model.add(base_model)
model.add(GaussianNoise(0.25))
model.add(GlobalAveragePooling2D())
model.add(Dense(256,activation='relu'))
model.add(BatchNormalization())
model.add(GaussianNoise(0.25))
model.add(Dropout(0.25))
model.add(Dense(1, activation='sigmoid'))
model.summary()

2022-11-04 05:50:46.671474: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-04 05:50:46.790235: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-04 05:50:46.790996: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-04 05:50:46.792909: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

43950080/43941136 [==============================] - 2s 0us/step
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
efficientnetb3 (Functional)  (None, 7, 7, 1536)        10783535  
_________________________________________________________________
gaussian_noise (GaussianNois (None, 7, 7, 1536)        0         
_________________________________________________________________
global_average_pooling2d (Gl (None, 1536)              0         
_________________________________________________________________
dense (Dense)                (None, 256)               393472    
_________________________________________________________________
batch_normalization (BatchNo (None, 256)               1024      
_________________________________________________________________
gaussian_noise_1 (GaussianNo (None, 256)               0         
_________________________________________________________

In [7]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy','AUC','Precision','Recall'])

In [8]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
es=EarlyStopping(patience=3,monitor='val_loss')
filepath='best_model.h5'
checkpoint = ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
history = model.fit(
    train_generator,
    epochs=15,
    validation_data=validation_generator,
    steps_per_epoch= 50,
    callbacks=checkpoint
    )

2022-11-04 05:50:55.291757: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/15


2022-11-04 05:51:04.579856: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


50/50 [==============================] - 36s 433ms/step - loss: 0.9254 - accuracy: 0.5750 - auc: 0.6294 - precision: 0.5588 - recall: 0.5876 - val_loss: 0.5348 - val_accuracy: 0.7444 - val_auc: 0.8457 - val_precision: 0.8728 - val_recall: 0.4975

Epoch 00001: val_accuracy improved from -inf to 0.74444, saving model to best_model.h5


/opt/conda/lib/python3.7/site-packages/keras/utils/generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 2/15
50/50 [==============================] - 17s 346ms/step - loss: 0.5912 - accuracy: 0.7200 - auc: 0.7984 - precision: 0.7075 - recall: 0.7500 - val_loss: 0.4956 - val_accuracy: 0.7578 - val_auc: 0.8398 - val_precision: 0.7370 - val_recall: 0.7075

Epoch 00002: val_accuracy improved from 0.74444 to 0.75778, saving model to best_model.h5
Epoch 3/15
50/50 [==============================] - 18s 355ms/step - loss: 0.6609 - accuracy: 0.7200 - auc: 0.7758 - precision: 0.7736 - recall: 0.7193 - val_loss: 0.3664 - val_accuracy: 0.8467 - val_auc: 0.9218 - val_precision: 0.8149 - val_recall: 0.8475

Epoch 00003: val_accuracy improved from 0.75778 to 0.84667, saving model to best_model.h5
Epoch 4/15
50/50 [==============================] - 16s 327ms/step - loss: 0.6391 - accuracy: 0.7000 - auc: 0.7837 - precision: 0.7429 - recall: 0.7027 - val_loss: 0.3716 - val_accuracy: 0.8322 - val_auc: 0.9146 - val_precision: 0.8285 - val_recall: 0.7850

Epoch 00004: val_accuracy did not improve from

In [9]:
model.evaluate(validation_generator)

225/225 [==============================] - 14s 62ms/step - loss: 0.3877 - accuracy: 0.8189 - auc: 0.9243 - precision: 0.8810 - recall: 0.6850


[0.3877110481262207,
 0.8188889026641846,
 0.9242975115776062,
 0.881028950214386,
 0.6850000023841858]

In [10]:
model.evaluate(train_generator)

2029/2029 [==============================] - 136s 67ms/step - loss: 0.3530 - accuracy: 0.8456 - auc: 0.9268 - precision: 0.8718 - recall: 0.8090


[0.3530002236366272,
 0.8456135988235474,
 0.9267920255661011,
 0.871767520904541,
 0.8090054392814636]

### The CNN model produced 82% validation accuracy. 

In [2]:
model.save('CNN for breast ultrasound.h5')